# OpenMC Scripts for Simulation of the PWR Fuel Cell

The following assumptions were made for generating data for Machine Learning phase of the project:

1. Temperature of the moderator was always lower or equal to the temperature of the fuel.
2. Temperature of the cladding was always equal to the temperature of the fuel.
3. Water density was calculated using iapws package (with pressure equal to 15 MPa and temperature equal to the temperature of the water used in simulations)
4. Thickness of the cladding was always equal to 0.06 cm

<br>
So, if you want to compare results of the OpenMC run with the result of the model, you will need to install additionally iapws package. This package is available in conda, so you can install it using:
<br>
conda install iapws
<br>

You also need to use the assumptions mentioned above in your OpenMC simulations, otherwise, the comarison with your ML model can be unreliable.

Importing necessary modules

Do not forget to install iapws package : conda install iapws

In [3]:
import os
import openmc
import numpy as np
from iapws import IAPWS97

The following function creates materials.xml for OpenMC simulations.
<br>
Input parameters:
* enr - enrichment of the uranium (weight percents)
* fuel_temperature - temperature of the fuel in K
* cladding_temperature - temperature of the cladding (zirconium) in K
* moderator_temperature - temperature of the moderator (water) in K
* fuel_density - density of the fuel, g/cm3
* cladding_density - density of the cladding (was equal to 6.6 and constant for all the simulations in the dataset), g/cm3
* moderator_density - density of the moderator, g/cm3
<br>
<br>
Output:
* List of the materials

In [4]:
# Functions for defining materials
def make_materials(enr=4.0, fuel_temperature=300.0, cladding_temperature=300.0, moderator_temperature=300.0,
                   fuel_density=10.0, cladding_density=6.6, moderator_density=1.0):
    # Defining uranium dioxide
    uo2 = openmc.Material(name="uo2")
    uo2.add_element('U', 1.0, enrichment=enr)
    uo2.add_element('O', 2.0)
    uo2.set_density('g/cm3', fuel_density)
    uo2.temperature = fuel_temperature
    
    # Zirconium
    zirconium = openmc.Material(name="zirconium")
    zirconium.add_element('Zr', 1.0)
    zirconium.set_density('g/cm3', cladding_density)
    zirconium.temperature = cladding_temperature

    # Water
    water = openmc.Material(name="h2o")
    water.add_element('H', 2.0)
    water.add_element('O', 1.0)
    water.set_density('g/cm3', moderator_density)
    water.temperature = moderator_temperature

    materials = openmc.Materials([uo2, zirconium, water])
    materials.export_to_xml()
    return materials

The following function creates the geometry.xml file for OpenMC simulations.
<br>
Input parameters:
* materials - list of the materials for simulations returned by function make_materials
* fuel_rad - radius of the fuel, cm
* clad_outer_rad - outer radius of the cladding, cm
* pitch - pitch of the cell, cm

In [5]:
def make_geometry(materials, fuel_rad=0.39, clad_outer_rad=0.46, pitch=1.26):
    fuel_outer_radius = openmc.ZCylinder(r=fuel_rad)
    clad_outer_radius = openmc.ZCylinder(r=clad_outer_rad)
    fuel_region = -fuel_outer_radius
    clad_region = +fuel_outer_radius & -clad_outer_radius
    
    fuel = openmc.Cell(name='fuel')
    fuel.fill = materials[0]
    fuel.region = fuel_region

    clad = openmc.Cell(name='clad')
    clad.fill = materials[1]
    clad.region = clad_region

    left = openmc.XPlane(-pitch/2, boundary_type='reflective')
    right = openmc.XPlane(pitch/2, boundary_type='reflective')
    bottom = openmc.YPlane(-pitch/2, boundary_type='reflective')
    top = openmc.YPlane(pitch/2, boundary_type='reflective')

    water_region = +left & -right & +bottom & -top & +clad_outer_radius

    moderator = openmc.Cell(name='moderator')
    moderator.fill = materials[2]
    moderator.region = water_region

    root_universe = openmc.Universe(cells=(fuel, clad, moderator))
    geometry = openmc.Geometry(root_universe)
    geometry.export_to_xml()

The following function creates settings.xml file for OpenMC simulations
<br>
Input parameters:
* batches - number of batches for simulations
* inactive - number of inactive cycles
* particles - number of particles (neutrons) to simulate in each batch

In [6]:
def make_settings(batches=150, inactive=30, particles=500):
    # Create a point source
    point = openmc.stats.Point((0, 0, 0))
    source = openmc.Source(space=point)
 
    settings = openmc.Settings()
    settings.source = source
    settings.batches = batches
    settings.inactive = inactive
    settings.particles = particles
    settings.temperature = {'method':'interpolation'}
    settings.export_to_xml()

The following function produces combines together previous three functions to produce geometry.xml, materials.xml and settings.xml files for OpenMC simulations.
<br>
Input parameters:
* enrichment - enrichment of the uranium (weight percents)
* pitch - pitch of the cell, cm
* frad - radius of the fuel, cm
* tfuel - temperature of the fuel in K
* tmoderator - temperature of the moderator (water) in K
* mod_dens - density of the moderator, g/cm3
* clad - thickness of the cladding, cm. Was equal to 0.06cm for all the simulations for data generation.
* batches - number of batches for simulations
* inactive - number of inactive cycles
* particles - number of particles (neutrons) to simulate in each batch

<br>
Cladding temperature was equal to fuel temperature (for simplicity) in all calculations


In [7]:
def make_openmc_input(enrichment, pitch, frad, tfuel, tmoderator, mod_dens, clad, batches, inactive, particles):
    materials = make_materials(enr=enrichment, fuel_temperature=tfuel, cladding_temperature=tfuel, moderator_density=mod_dens, moderator_temperature=tmoderator)
    make_geometry(materials=materials, fuel_rad=frad, clad_outer_rad=frad + clad, pitch=pitch)
    make_settings(batches=batches, inactive=inactive, particles=particles)

The following function removes the results of the previous simulations when simulations run in a row.

In [8]:
def clean_data():
    for file in os.listdir("."):
        if file.endswith(".h5"):
            print("File {} was removed".format(os.path.abspath(file)))
            os.remove(os.path.abspath(file))
        if file.endswith(".xml"):
            print("File {} was removed".format(os.path.abspath(file)))
            os.remove(os.path.abspath(file))

The following function extracts keff and standard deviation from the output file when OpenMC simulations finished.
Input:
* batches - number of batches used in the simulations
<br>
Output:
* keff - effective multiplication factor
* std - standard deviation

In [9]:
def get_keff_std(batches):
    sp = openmc.StatePoint('statepoint.' + str(batches) + '.h5')
    keff = sp.keff.nominal_value
    std =  sp.keff.std_dev
    sp.close
    return keff, std

The following function runs OpenMC simulations with one or more threads
<br>
Input parameters:
* nthreads - number of threads

In [10]:
def run_openmc(nthreads=None, output=False):
    if nthreads is None:
        openmc.run(output=output)
    else:
        openmc.run(threads=nthreads, output=output)

## Examples

Single OpenMC run

In [11]:
enrichment = 4.5
pitch = 1.26
frad = 0.45
tfuel = 900 
tmoderator = 600
# Defining water density using IAPWS97 module, with given temperature and pressure
water = IAPWS97(T=tmoderator, P=15.)
mod_dens = round(water.rho * 0.001, 3)
clad = 0.06
batches = 150
inactive = 50
particles = 10000
nthreads = 30

make_openmc_input(enrichment, pitch, frad, tfuel, tmoderator, mod_dens, clad, batches, inactive, particles)
run_openmc(nthreads=nthreads, output=True)
keff, std = get_keff_std(batches=batches)

/home/dzianis/anaconda3/envs/openmc-env/lib/python3.11/site-packages/openmc/source.py:390: FutureWarning: This class is deprecated in favor of 'IndependentSource'
  warnings.warn("This class is deprecated in favor of 'IndependentSource'", FutureWarning)


                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [12]:
print(keff)

1.3026228981871255


In [13]:
print(std)

0.0009491550850895128


Print standard deviation in pcm:

In [14]:
print(" Standard deviation is {:} pcm".format(int(std*100000)))

 Standard deviation is 94 pcm
